In [ ]:
import itertools

from copy import deepcopy



In [ ]:
def overlap(a, b, min_length=3):

    """ Return length of longest suffix of 'a' matching

        a prefix of 'b' that is at least 'min_length'

        characters long.  If no such overlap exists,

        return 0. """

    start = 0  # start all the way at the left

    while True:

        start = a.find(b[:min_length], start)  # look for b's prefix in a

        if start == -1:  # no more occurrences to right

            return 0

        # found occurrence; check for full suffix/prefix match

        if b.startswith(a[start:]):

            return len(a)-start

        start += 1  # move just past previous match

def pick_maximal_overlap(reads, k, kmers):
    reada, readb = None, None
    best_olen = 0
    for read in reads:
        current_suffix = read[len(read)-k:]
        reads_with_kmer = deepcopy(kmers[current_suffix])
        reads_with_kmer.discard(read)
        for read_with_kmer in reads_with_kmer:
            olen = overlap(read,read_with_kmer,k)
            if olen > best_olen:
               reada, readb = read, read_with_kmer 
               best_olen = olen
    return reada, readb, best_olen

    

def greedy_scs(reads, k):
    kmers = {}

    
    for read in reads:
        for i in range(len(read) - k + 1):  # for each k-mer
            current_kmer = read[i:i+k]
            if current_kmer not in kmers:
                kmers[current_kmer] = set([read])
            else:
                kmers[current_kmer].update([read])
                
    read_a, read_b, olen = pick_maximal_overlap(reads, k, kmers)
    
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        
        for key, value in kmers.items():
            for val in value.copy():
                if val == read_a or val == read_b:
                    value.discard(val)
                    
        read = read_a + read_b[olen:]
        for i in range(len(read) - k + 1):  # for each k-mer
            current_kmer = read[i:i+k]
            if current_kmer not in kmers:
                kmers[current_kmer] = set([read])
            else:
                kmers[current_kmer].update([read])
                
        read_a, read_b, olen = pick_maximal_overlap(reads, k, kmers)
    return ''.join(reads)            


seqs, quals = readFastq('example.fq')

final_scs = greedy_scs(seqs, 10)